In [0]:
from functools import total_ordering

In [0]:
class Heap(list):
    """继承 python 内置的 list 类型，并添加 heap_size 属性"""
    def __init__(self, array, heap_size=None):
        list.__init__(self, array)
        self.heap_size = len(array) if heap_size is None else heap_size

## 6.1 堆 

- 堆是一个数组，可以被看成一个近似的完全二叉树（除了最底层，该树应该是完全充满的）
- 示意图  
<img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191211201233.png width = 600>
- $A\left[ {1 \ldots A.length} \right]$ 中可能都含有元素，但只有 $A\left[ {1 \ldots A.heap\_size} \right]$ 中存放的是有效元素

#### **获取给定节点的父节点，左孩子结点和右孩子结点** 


- 常用编程语言的下标从 0 开始，除了代码部分，则默认数组下标从 1 开始
- 可通过左移或右移操作来加快计算速度
- python 的移位操作优先级较低，注意加括号以避免出错

In [0]:
def parent(i):
    return (i-1) >> 1

def left(i):
    return (i << 1) + 1

def right(i):
    return (i << 1) + 2

In [0]:
i = 3
print("the parent of i is {}, the left child of i is {}, the right child of i is {}". format(parent(i), left(i), right(i)))

the parent of i is 1, the left child of i is 7, the right child of i is 8


#### 最大堆与最小堆

- 最大堆的性质
    - 对于除了根节点以外的所有节点 $i$ 都要满足：$$A\left[ {PARENT\left( i \right)} \right] \ge A\left[ i \right]$$
    
    - 即最大堆中的最大元素存放在根节点中，且在任一子树中，这一性质也成立


- 最小堆的性质
    - 对于除了根节点以外的所有节点 $i$ 都要满足：$$A\left[ {PARENT\left( i \right)} \right] \le A\left[ i \right]$$
    
    - 即最小堆中的最小元素存放在根节点中，且在任一子树中，这一性质也成立

#### 堆的应用

- 排序
- 优先队列

#### 堆的性质

- 包含 $n$ 个元素的堆高度为 $\Theta(lg(n))$
- 堆结构上的一些基本操作至多与堆的高度成正比，即时间复杂度为 $O(lg(n))$
- 最大堆的最小元素位于叶节点中
- 已排好序的数组是一个最小堆
- 当用数组表示储存 $n$ 个元素的堆时，叶结点的下标分别为 $\left\lfloor {{n \over 2}} \right\rfloor  + 1,\left\lfloor {{n \over 2}} \right\rfloor  + 2, \cdots ,n$, 即叶节点的数目为 $\left\lceil {{n \over 2}} \right\rceil$
- 当只含有一个元素时，堆的高度为 0

## 6.2 维护堆的性质

- 在调用 MAX-HEAPIFY 假设：
    - 根结点为 LEFT($i$) 和 RIGHT($i$) 的二叉树都是最大堆
    - $A[i]$ 可能小于其孩子节点

### 代码实现

In [0]:
def max_heapify(A, i):
    l = left(i)
    r = right(i)
    largest = i
    if l < A.heap_size and A[l] > A[largest]:
        largest = l
    if r < A.heap_size and A[r] > A[largest]:
        largest = r
    if largest != i:
        A[i], A[largest] = A[largest], A[i]
        max_heapify(A, largest)

In [0]:
A = Heap([16, 4, 10, 14, 7, 9, 3, 2, 8, 1])
max_heapify(A, 1)
print(A)

[16, 14, 10, 8, 7, 9, 3, 2, 4, 1]


### 程序执行示意图

<img src="https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191211201929.png" width = 600>

###  复杂度分析

- 调用 $A[i], A[LEFT(i)], A[RIGHT(i)]$ 的时间代价为 $\Theta(1)$
- 子树的规模最大为 $ 2n/3 $
  - 发生在最底层恰好半满的时候
      - 设树高为 $h$，则除最底层个，包含的元素数为：
      $$1 + 2 + {2^2} +  \cdots  + {2^{h - 1}} = {{{2^h} - 1} \over {2 - 1}} = {2^h} - 1$$
      - 左右子树各占 $(2^h-1)/2 = 2^{h-1} - 1/2$
      - 最后一层半满时，元素数为 $2^{h-1}$
      - 最大子树的规模为：
      $${{{2^{h - 1}} + {{\rm{2}}^{h - 1}} - 1/2} \over {{2^{h - 1}} + {2^h} - 1}} \approx {2 \over 3}$$
- 运行时间的递推式为：$$T(n) \le T(2n/3) + \Theta(1)$$

由主定理可得，运行时间为 $O\left( {\lg n} \right)$，即 $O(h)$

###  练习

####  6.2-2 MINI-HEAPIFY

In [0]:
def mini_heapify(A, i):
    l = left(i)
    r = right(i)
    smallest = i
    if l < A.heap_size and A[l] < A[smallest]:
        smallest = l
    if r < A.heap_size and A[r] < A[smallest]:
        smallest = r
    if smallest != i:
        A[smallest], A[i] = A[i], A[smallest]
        mini_heapify(A, smallest)

In [0]:
A = Heap([1, 10, 3, 4, 5, 6, 7, 8, 9, 10])
mini_heapify(A, 1)
print(A)

[1, 4, 3, 8, 5, 6, 7, 10, 9, 10]


### 6.2-5 用循环控制结构重写 MAX-HEAPIFY

In [0]:
def max_heapify_loop(A, i):
    while True:
        l = left(i)
        r = right(i)
        largest = i
        if l < A.heap_size and A[l] > A[largest]:
            largest = l
        if r < A.heap_size and A[r] > A[largest]:
            largest = r
        if largest != i:
            A[i], A[largest] = A[largest], A[i]
            i = largest
        else:
            break

In [0]:
A = Heap([16, 4, 10, 14, 7, 9, 3, 2, 8, 1])
max_heapify_loop(A, 1)
print(A)

[16, 14, 10, 8, 7, 9, 3, 2, 4, 1]


## 6.3 建堆

- 子数组 $A\left( {\left\lfloor {n/2} \right\rfloor  + 1 \ldots n} \right)$ 中的元素都是树的叶节点
- 每个叶节点都可以看成只包含一个元素的堆
- 因此， BUILD-MAX-HEAP 对树中的其他节点都调用一次 MAX-HEAPIFY 即可

###  代码实现

In [0]:
def build_max_heap(A):
    A.heap_size = len(A)
    for i in reversed(range(len(A)//2)):
        max_heapify(A, i)

In [0]:
A = Heap([4, 1, 3, 2, 16, 9, 10, 14, 8, 7])
build_max_heap(A)
print(A)

[16, 14, 10, 8, 7, 9, 3, 2, 4, 1]


###  示意图

<img src="https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191211211919.png" width=600>

### 复杂度分析

- 高度为 $h$ 的子树至多有 $\lceil n/2^{h+1}\rceil$ 个(练习 6.3-3)
- 高度为 $h$ 的子树的根节点调用 MAX-HEAPIFY 的时间复杂度为 $O(h)$
- 总的时间复杂度为：
$$\sum\limits_{h = 0}^{\left\lfloor {\lg n} \right\rfloor } {\left\lceil {{n \over {{2^{h + 1}}}}} \right\rceil } O\left( h \right) = O\left( {n\sum\limits_{h = 0}^{\left\lfloor {\lg n} \right\rfloor } {{h \over {{2^h}}}} } \right)$$
结合公式 (A.8) 可得：
$$O\left( {n\sum\limits_{h = 0}^{\left\lfloor {\lg n} \right\rfloor } {{h \over {{2^h}}}} } \right) = O\left( {n \times {{{1 \over 2}} \over {{{\left( {1 - {1 \over 2}} \right)}^2}}}} \right) = O\left( n \right)$$
由此可得，建堆的时间复杂度为 $O(n)$

### 练习

#### 6.3 -3
证明对于一个包含 $n$ 个元素的堆，至多有 $\lceil n/2^{h+1}\rceil个子树$

1. 包含 $n$ 个元素的堆,其叶子节点的数目为 $\lceil n/2 \rceil$
2. 对于堆的子树而言，其叶子节点即为堆的叶子节点，而且高度为 $h$ 的各个子树的叶子节点不会有重复。假设高度为 $h$ 的子树一共有 $k$ 个，则有 $ k - 1 $ 个子树的叶子节点为 $2^h$ 而一个子树的叶子节点在 $[1, 2^h]$ 之间, 设为 $x$  
由 1，2 可得：
$$(k-1)2^h + x \le k * 2^h \le \lceil n/2 \rceil$$
即：
$$k \le {{\lceil n/2 \rceil} \over 2^h} \le \left\lceil {{{\left\lceil {{n \over 2}} \right\rceil } \over {{{\rm{2}}^h}}}} \right\rceil  $$
由书中公式 (3.4) 可得：
$$k \le \left\lceil {{n \over {{2^{h + 1}}}}} \right\rceil $$
证毕

## 6.4 堆排序算法

- 基本思路  
  1. 先根据数组建最大堆
  2. 从最大堆顶取出最大的元素与堆的最后一个元素交换，然后将堆的长度减 1，并维护堆的性质
  3. 重复 2 中的步骤，直到最大堆中的元素数目等于 1

### 代码实现

In [0]:
def heapsort(A):
    build_max_heap(A)
    while A.heap_size > 1:
        A[0], A[A.heap_size - 1] = A[A.heap_size - 1], A[0]
        A.heap_size = A.heap_size - 1
        max_heapify(A, 0)

In [0]:
A = Heap([4, 1, 3, 2, 16, 9, 10, 14, 8, 7])
heapsort(A)
print(A)

[1, 2, 3, 4, 7, 8, 9, 10, 14, 16]


### 代码执行过程

<img src="https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191212204631.png" width=600>

### 复杂度分析

- BUILD-MAX-HEAP 的时间复杂度为 $O(n)$
- MAX-HEAPIFY 的时间复杂度为 $O(lgn)$，会被调用 $n-1$ 次
- 整体的时间复杂变为 $O(nlg(n))$

## 6.5 优先队列

- 优先队列是一种用来维护由一组元素组成的集合 $S$ 的数据结构，其中每种元素都有一个相关的值，称为关键字(key)

### 最大优先队列

- 支持如下操作  
  - INSERT(S, x): 将元素 $x$ 插入到集合 $S$ 中
  - MAXIMUN(S,x): 返回 $S$ 中具有最大键值的元素
  - EXTRACT-MAX(S): 去掉并返回 $S$ 中具有最大键值的元素
  - INCREASE-KEY(S, x, k)： 将元素 $x$ 的关键值增加到 $k$，要求 $k$ 不小于 $x$ 原关键字的值
- 应用实例
  - 计算机中的作业调度，可以随时使用 INSERT 将新的作业插入进来

### 最小优先队列

- 包括 INSERT, MINIMUN, EXTRACT_MIN 和 DECREASE-KEY 操作
- 应用实例
  - 基于事件驱动的模拟器
    - 事件必须按照发生的时间顺序进行模拟，因为某一事件的模拟结果可能触发对其它事件的模拟

### 最大优先队列常用操作的代码实现
- 一般需要在堆中储存对应对象的句柄

#### HEAP-MAXIMUN

In [0]:
def heap_maximun(A):
  if A.heap_size < 1:
    raise Exception("Heap underflow")
  return A[0]

In [0]:
A = Heap([16, 14, 10, 8, 7, 9, 3, 2, 4, 1])
heap_maximun(A)

16

#### HEAP-EXTRACT-MAX

In [0]:
def heap_extract_max(A):
  if A.heap_size < 1:
    raise Exception("Heap underflow")
  max = A[0]
  A[0] = A[A.heap_size-1]
  A.heap_size -= 1
  max_heapify(A, 0)
  return max

In [0]:
A = Heap([16, 14, 10, 8, 7, 9, 3, 2, 4, 1])
print(heap_extract_max(A))
print(A[:A.heap_size])

16
[14, 8, 10, 4, 7, 9, 3, 2, 1]


#### HEAP-INCREASE-KEY

In [0]:
def heap_increase_key(A, i, key):
  if not(0<= i <A.heap_size):
    raise Exception("index out of range!")
  if key < A[i]:
    raise Exception("new key is smaller than current key!")
  A[i] = key
  while i > 0 and A[parent(i)] < A[i]:
    A[i], A[parent(i)] = A[parent(i)], A[i]
    i = parent(i)

In [0]:
A = Heap([16, 14, 10, 8, 7, 9, 3, 2, 4, 1])
heap_increase_key(A, 8, 15)
print(A)

[16, 15, 10, 14, 7, 9, 3, 2, 8, 1]


- 执行过程示意图  
<img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191213110736.png width=600>

#### MAX-HEAP-INSERT
- 借助 HEAP-INCREASE-KEY 来实现


In [0]:
def max_heap_insert(A, key):
  A.heap_size = A.heap_size + 1
  if A.heap_size > len(A):
    A.append(-float('inf'))
  else:
    A[A.heap_size-1] = -float('inf')
  heap_increase_key(A, A.heap_size-1, key)

In [0]:
A = Heap([16, 14, 10, 8, 7, 9, 3, 2, 4, 1])
max_heap_insert(A, 15)
print(A)

[16, 15, 10, 8, 14, 9, 3, 2, 4, 1, 7]


### 练习题

#### 6.5-3 最小优先队列常用的操作

##### HEAP-MINIMUM

In [0]:
def heap_minimun(A):
  if A.heap_size < 1:
    raise Exception("Heap underflow")
  return A[0]

##### HEAP-EXTRACT-MIN

In [0]:
def heap_extract_min(A):
  if A.heap_size < 1:
    raise Exception("heap underflow")
  min = A[0]
  A.heap_size -= 1
  A[0] = A[A.heap_size]
  mini_heapify(A, 0)
  return min

In [0]:
A = Heap([1, 2, 3, 4, 5, 6, 7, 8, 9])
print(heap_extract_min(A))
print(A[:A.heap_size])

1
[2, 4, 3, 8, 5, 6, 7, 9]


##### HEAP-DECREASE-KEY

In [0]:
def heap_decrease_key(A, i, key):
  if not(0 <= i < A.heap_size):
    raise Exception('List out of range!')
  if A[i] < key:
    raise Exception('new key is bigger than current key!')
  A[i] = key
  while i > 0 and A[parent(i)] > key:
    A[i] = A[parent(i)]
    i = parent(i)
  A[i] = key

In [0]:
A = Heap([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
heap_decrease_key(A, 4, 0)
print(A)

[0, 1, 3, 4, 2, 6, 7, 8, 9, 10]


##### MINI-HEAP-INSERT

In [0]:
def mini_heap_insert(A, key):
  A.heap_size += 1
  if A.heap_size > len(A):
    A.append(float('inf'))
  else:
    A[A.heap_size - 1] = float('inf')
  heap_decrease_key(A, A.heap_size-1, key)

In [0]:
A = Heap([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
mini_heap_insert(A, 2)
print(A)

[1, 2, 3, 4, 2, 6, 7, 8, 9, 10, 5]


#### 6.5-6 一次赋值完成 HEAP-INCREASE-KEY 操作

In [0]:
def heap_increase_key_one_assignment(A, i, key):
  if not(0<= i <A.heap_size):
    raise Exception("index out of range!")
  if key < A[i]:
    raise Exception("new key is smaller than current key!")
  A[i] = key
  while i > 0 and A[parent(i)] < A[i]:
    A[i] = A[parent(i)]
    i = parent(i)
  A[i] = key

In [0]:
A = Heap([16, 14, 10, 8, 7, 9, 3, 2, 4, 1])
heap_increase_key_one_assignment(A, 8, 15)
print(A)

[16, 14, 10, 15, 7, 9, 3, 2, 8, 1]


#### 6.5-8 HEAP-DELETE(A, i)

##### 最大堆

In [0]:
def max_heap_delete(A, i):
  if not(0<= i <=A.heap_size):
    raise Exception('index out of range')
  A.heap_size -= 1
  A[i] = A[A.heap_size]
  max_heapify(A, i)

In [0]:
A = Heap([10, 9, 8, 7, 6, 5, 4, 3, 2, 1])
max_heap_delete(A, 1)
print(A[:A.heap_size])

[10, 7, 8, 3, 6, 5, 4, 1, 2]


##### 最小堆

In [0]:
def mini_heap_delete(A, i):
  if not(0<= i <=A.heap_size):
    raise Exception('index out of range')
  A.heap_size -= 1
  A[i] = A[A.heap_size]
  mini_heapify(A, i)

In [0]:
A = Heap([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
mini_heap_delete(A, 1)
print(A[:A.heap_size])

[1, 4, 3, 8, 5, 6, 7, 10, 9]


#### 6.5-9 在 $O(nlg(k))$ 内归并 $k$ 个有序链表

##### 基本思路

将 $k$ 个列表的第一个元素取出，建成最小堆